In [1]:
import re
import shutil
import pandas as pd
from pandas import DataFrame, Series
import github3
from github3 import login
from git import Repo, GitCommandError
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
from github3.repos.branch import Branch
from github3.null import NullObject
from schematics.models import Model, BaseType
from schematics.types import StringType, IntType, BooleanType
from schematics.types.compound import ListType, DictType
from schematics.exceptions import ValidationError
%cd ~/Documents/projects/stitch/python/
from stitch.core.stitch_frame import StitchFrame
%cd ~/Documents/projects/nerve/python/
from nerve.core.model import Nerve
from nerve.core.client import Client
from nerve.core.utils import *
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.metadata import Metadata
from nerve.core.utils import execute_subprocess, get_status
from nerve.spec.specifications import *
from nerve.spec.validators import *
from nerve.spec.traits import *
from nerve.spec import traits
from nerve.test.test_suite import *
%cd ~/Documents/projects/nerve

/Users/alex/Documents/projects/stitch/python
/Users/alex/Documents/projects/nerve/python
/Users/alex/Documents/projects/nerve


In [2]:
def stat(nerve):
    stat = nerve.status(status_asset_types=['deliverable', 'nondeliverable'])
    data = DataFrame(list(stat)).applymap(lambda x: x.data)
    data = StitchFrame(data).flatten(prefix=False).to_dataframe().T
    return data

config_path = '/Users/alex/Documents/projects/nerve/python/nerve/resources/nerverc.yml'
nerve = Nerve(config_path)
proj = os.path.join(nerve['project-root'], nerve['project']['project-name'])
info = nerve._get_info(None, None, {})
cc = info.client_conf

In [3]:
nerve.delete(True, True)

True

In [3]:
nerve.delete(False, True)

True

In [3]:
nerve.delete(True, False)

True

In [4]:
nerve.create(notes='some note about a thing')

True

In [4]:
nerve.clone()

True

In [5]:
%%bash
cd /tmp/ntr001
echo vasc v001 > maya001/ntr001_maya001_vasc_v001.mb
echo kidney v001 > maya001/ntr001_maya001_kdny_v001.mb
tree

.
├── geo001
├── maya001
│   ├── ntr001_maya001_kdny_v001.mb
│   └── ntr001_maya001_vasc_v001.mb
├── proj001_meta.yml
└── vol001

3 directories, 3 files


In [6]:
stat(nerve)

,0,1
asset-id,2044dbce-2bd1-4f8f-87ff-42...,7c5dfbb0-d6e3-437a-a9fb-bf...
asset-name,ntr001_maya001_kdny_v001,ntr001_maya001_vasc_v001
asset-type,nondeliverable,nondeliverable
data,None,None
notes,,
project-id,None,None
project-name,ntr001,ntr001
specification,maya001,maya001
url,None,None


In [7]:
self = nerve
nondeliverables = self.status(name=info.name,
    status_asset_types=['nondeliverable'], **info.config)
nondeliverables = list(nondeliverables)
for non in nondeliverables:
    non.get_traits()
    non.write(validate=False)

In [3]:
# publish nondeliverables
lfs = GitLFS(info.path, environment=info.env)
local = Git(info.path, branch=info.branch, environment=info.env)

In [3]:
# push non-deliverables to user-branch
local.add([x.metapath for x in nondeliverables])
local.add([x.datapath for x in nondeliverables])
names = [x['asset-name'] for x in nondeliverables]
local.commit('NON-DELIVERABLES: ' + ', '.join(names))

In [4]:
url = info.lfs_url
branch = info.branch
cmd = 'git lfs push {remote} {branch}'.format(remote=url, branch=branch)
execute_subprocess(cmd, proj, environment=info.env)

[]

In [4]:
lfs.push(info.lfs_url, info.branch)
# lfs.remove_prepush()
# local.push(info.branch)

In [5]:
nerve._publish_nondeliverables(info)
nerve._update_local(info)

In [29]:
nerve.publish(notes='create vol001_v001; if you can read this, it probably worked', verbosity=2)